# GC-IGR Graphing Worksheet

## Purpose:

This jupyter notebook will aid in choosing a genome, download the relevant files from NCBI, extract intergenic regions, match IGRs to known Rfam annotations, and then use support vector machine classifiers to select IGRs for further analysis.  

### Necessary Imports and Configuration

In [1]:
%cd '/home/jovyan/work'

import sys
import os
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import numpy as np
import shutil
import tarfile
import pickle
from sklearn import svm
from sqlalchemy import or_
from ipywidgets import interactive
from Bio import SeqIO, SeqRecord

from src.data.make_dataset import extract_igrs, annotate_igrs, download_genome
from src.visualization.visualize import graph_genome, graph_layout
from src.data.rfam_db import rfam_session, Genome

py.io.orca.config.use_xvfb = True
pd.set_option('display.max_columns', 60)

/home/jovyan/work


## Step 1: Review Bacterial and Archaeal Genomes with Rfam annotations and select UniprotID (upid)

In [7]:
# Create a connection to local or remote Rfam Database
session = rfam_session()

# Get list of bacterial and archaeal genomes and save them in Genome DF
genome_query = session.query(Genome).filter(or_(Genome.kingdom=='archaea', Genome.kingdom=='bacteria'))

# If necessary filter for completely assembled genomes.
genome_query = genome_query.filter(Genome.assembly_level == 'complete-genome')
genome_list = genome_query.all()
session.close()
genome_df = pd.read_sql_query(genome_query.statement, genome_query.session.bind)

# Display the the genomes numbered 0-9 from the above criteria. 
genome_df.iloc[0:10]

upid     assembly_acc  assembly_version wgs_acc  wgs_version  \
0  UP000000212  GCA_000317975.2                 2    None          NaN   
1  UP000000229  GCA_000016565.1                 1    None          NaN   
2  UP000000230  GCA_000016325.1                 1    None          NaN   
3  UP000000231  GCA_000016345.1                 1    None          NaN   
4  UP000000233  GCA_000013785.1                 1    None          NaN   
5  UP000000235  GCA_000016425.1                 1    None          NaN   
6  UP000000238  GCA_000012985.1                 1    None          NaN   
7  UP000000239  GCA_000055785.1                 1    None          NaN   
8  UP000000242  GCA_000016605.1                 1    None          NaN   
9  UP000000243  GCA_000014305.1                 1    None          NaN   

  assembly_name   assembly_level   study_ref  \
0    ASM31797v2  complete-genome    PRJEB544   
1     ASM1656v1  complete-genome  PRJNA17457   
2     ASM1632v1  complete-genome  PRJNA17461   
3     ASM1634v1  complete-genome  PRJNA16679   
4     ASM1378v1  complete-genome  PRJNA16817   
5     ASM1642v1  complete-genome  PRJNA16342   
6     ASM1298v1  complete-genome  PRJNA16064   
7     ASM5578v1  complete-genome  PRJNA12636   
8     ASM1660v1  complete-genome  PRJNA17447   
9     ASM1430v1  complete-genome  PRJNA17153   

                                         description  total_length  \
0  ASM31797v2 assembly for Carnobacterium maltaro...       3650416   
1   ASM1656v1 assembly for Pseudomonas mendocina ymp       5072807   
2        ASM1632v1 assembly for Enterobacter sp. 638       4676461   
3  ASM1634v1 assembly for Polynucleobacter asymbi...       2159490   
4  ASM1378v1 assembly for Pseudomonas stutzeri A1501       4567418   
5  ASM1642v1 assembly for Salinispora tropica CNB...       5183331   
6  ASM1298v1 assembly for Hahella chejuensis KCTC...       7215267   
7  ASM5578v1 assembly for Chromohalobacter salexi...       3696649   
8  ASM1660v1 assembly for Metallosphaera sedula D...       2191517   
9  ASM1430v1 assembly for Streptococcus suis 05ZYH33       2096309   

   ungapped_length  circular  ncbi_id  \
0          3650416       0.0  1234679   
1          5072807       0.0   399739   
2          4676461       0.0   399742   
3          2159490       0.0   312153   
4          4567418       0.0   379731   
5          5183331       0.0   369723   
6          7215267       0.0   349521   
7          3696649       0.0   290398   
8          2191517       0.0   399549   
9          2096309       0.0   391295   

                              scientific_name common_name   kingdom  \
0         Carnobacterium maltaromaticum LMA28        None  bacteria   
1                   Pseudomonas mendocina ymp        None  bacteria   
2                        Enterobacter sp. 638        None  bacteria   
3  Polynucleobacter asymbioticus QLW-P1DMWA-1        None  bacteria   
4                  Pseudomonas stutzeri A1501        None  bacteria   
5                 Salinispora tropica CNB-440        None  bacteria   
6                Hahella chejuensis KCTC 2396        None  bacteria   
7        Chromohalobacter salexigens DSM 3043        None  bacteria   
8              Metallosphaera sedula DSM 5348        None   archaea   
9                  Streptococcus suis 05ZYH33        None  bacteria   

   num_rfam_regions  num_families  is_reference  is_representative  \
0               134            40             0                  1   
1               166            63             0                  1   
2               322           114             0                  1   
3                55            21             0                  1   
4               158            61             0                  1   
5               138            30             0                  1   
6               123            28             0                  1   
7                96            21             0                  1   
8                57            

## Step 2: Enter the upid of the genome of interest and graph genome.

Look up the Uniprot ID (upid) for a genome of interest here: http://rfam.xfam.org/search?q=entry_type:%22Genome%22 and enter it below for the upid:



In [2]:
upid = 'UP000001174'

session = rfam_session()
genome =  session.query(Genome).get(upid)
session.close()
download_genome(genome)
igr_df = extract_igrs(genome, igr_length_cutoff=1)
annotated_df = annotate_igrs(genome, igr_df)
scatter_plots = graph_genome(annotated_df)
layout = graph_layout(genome)
fig = go.FigureWidget(data=scatter_plots, layout=layout)
fig

ValidationError: Failed to find tag 'eSummaryResult' in the DTD. To skip all tags that are not represented in the DTD, please call Bio.Entrez.read or Bio.Entrez.parse with validate=False.

## Step 3: SVM Selection of Genomic Regions Enriched for ncRNAs

If necessary, modify the SVM selection hyperparameters (class_weight_mod, gamma_exp, and c_exp) using the sliders below.


In [4]:
# Define function for interactive modification of selection parameters. 

def prepare_selection(annotated_df):
    y = (annotated_df['category'] != 'No Known RNA') & (annotated_df['category'] != 'sRNA') 
    total_igrs = len(y)
    total_knowns = y.sum()
    total_unknowns = total_igrs - total_knowns 
    
    return (y, total_igrs, total_knowns, total_unknowns)

def build_interactive_fn (annotated_df):
    
    y, total_igrs, total_knowns, total_unknowns = prepare_selection(annotated_df)    

    def interactive_fn(class_weight_mod=0.5,  c_exp=2, gamma_exp=-2,):
        class_weight = {False: total_knowns / total_igrs, True: (total_unknowns / total_igrs * class_weight_mod)}
        svm_clf = svm.SVC(C=10**c_exp, class_weight=class_weight, gamma=10**(gamma_exp), random_state=0)
        svm_clf.fit(annotated_df.loc[:, ["gc", "log_length"]], y)
        selection = pd.Series(svm_clf.predict(annotated_df.loc[:, ["gc", "log_length"]]))
        scatter_plots = graph_genome(annotated_df, selection=selection)
        fig = go.FigureWidget(data=scatter_plots, layout=layout)
        display(fig)
    
    return interactive_fn


interactive_fn = build_interactive_fn(annotated_df)
interactive_plot = interactive(interactive_fn, class_weight_mod=(0.05, 2.0, 0.05), gamma_exp=(-5, 5, 0.25), c_exp=(-5,5,0.25))
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='class_weight_mod', max=2.0, min=0.05, step=0.05), Fl…

## Step 4: Finalize Selection and Build Blast Script/Data Tarfile

After finalizing the selection in the interactive graph above, execute the following two blocks of code to extract the selected intergenic regions, and prepare a tarfile with the collection of data and scripts necessary for blast analysis. 

In [5]:
# Extract the values from the interactive plot
class_weight_mod = interactive_plot.kwargs["class_weight_mod"]
c_exp = interactive_plot.kwargs["c_exp"]
gamma_exp = interactive_plot.kwargs["gamma_exp"]

output_folder="data/interim/{}/selection_{}_{}_{}".format(genome.assembly_acc, class_weight_mod, c_exp, gamma_exp)
if not os.path.exists(output_folder + '/data_files'):
    os.makedirs(output_folder + '/data_files')

# Re-create the selection
y, total_igrs, total_knowns, total_unknowns = prepare_selection(annotated_df)    
class_weight = {False: total_knowns / total_igrs, True: (total_unknowns / total_igrs * class_weight_mod)}
svm_clf = svm.SVC(C=10**c_exp, class_weight=class_weight, gamma=10**gamma_exp, probability=True, random_state=0)
svm_clf.fit(annotated_df.loc[:, ["gc", "log_length"]], y)

# Save the selection classifier to a pickle                                                          
svm_pickle = pickle.dumps(svm_clf)
with open("{}/data_files/svmclf.pickle".format(output_folder,genome.assembly_acc, class_weight_mod, c_exp, gamma_exp), 'wb') as svm_pickle_file:
    svm_pickle_file.write(svm_pickle)
selection = pd.Series(svm_clf.predict(annotated_df.loc[:, ["gc", "log_length"]]))
                                                          
# Save a graph of the genome.
scatter_plots = graph_genome(annotated_df, selection=selection)
layout = graph_layout(genome)
fig = go.FigureWidget(data=scatter_plots, layout=layout)
fig.write_image("{}/data_files/{}_plot.svg".format(output_folder,genome.scientific_name.replace(' ','_')))
py.io.write_json(fig, "{}/data_files/{}_json.plotly".format(output_folder,genome.scientific_name.replace(' ','_')))
                                                          
selected_unknowns = selection & (annotated_df['category'] == 'No Known RNA')

# Save a fasta file with all the selected IGRs
selected_igr_list = [SeqRecord.SeqRecord(row.sequence, id=("{}/{}-{}".format(row.accession, row.start +1, row.end))) 
                     for row in annotated_df.loc[selected_unknowns, ["accession","start","end","sequence"]].itertuples()]

if not os.path.exists(output_folder + '/igr_fastas'):
    os.makedirs(output_folder + '/igr_fastas')

for igr in selected_igr_list:
    outputfilename = "{}/igr_fastas/{}.fasta".format(output_folder, igr.id.replace('/','_'))
    SeqIO.write(igr, outputfilename, "fasta")
    
annotated_df.to_csv("{}/data_files/annotated_igrs.csv".format(output_folder), index=False)

Number of known IGRs included:   31 (88.6%)
Number of unknown IGRs included: 108 (7.1%)
Fold Enrichment:  9.67


In [6]:
if not os.path.exists(output_folder + '/scripts'):
    os.makedirs(output_folder + '/scripts')

shutil.copy('src/shell/blast_source_template.sh', '{}/scripts/blast_source.sh'.format(output_folder))
shutil.copy('src/shell/blast_run_template.sh', '{}/blast_run.sh'.format(output_folder))

if not os.path.exists(output_folder + '/blast_xml'):
    os.makedirs(output_folder + '/blast_xml')
if not os.path.exists(output_folder + '/output'):
    os.makedirs(output_folder + '/output')

with open("{}/scripts/blast_jobfile.sh".format(output_folder), 'w') as jobfile:
    for igr in selected_igr_list:
        fasta_filename = "igr_fastas/{}.fasta".format(igr.id.replace('/','_'))
        xml_filename =  "blast_xml/{}.xml".format(igr.id.replace('/','_'))
        jobfile.write("source scripts/blast_source.sh; $BLASTCMD {} > {}\n".format(fasta_filename, xml_filename))
        
with tarfile.open("data/export/{}_{}_selection_{}_{}_{}_blastdata.tar.gz".format('_'.join(genome.scientific_name.split(' ')[0:2]), genome.assembly_acc, class_weight_mod, c_exp, gamma_exp), "w:gz") as tar:
    tar.add(output_folder, arcname="{}_{}_selection_{}_{}_{}_blastdata".format('_'.join(genome.scientific_name.split(' ')[0:2]), genome.assembly_acc, class_weight_mod, c_exp, gamma_exp))